In [1]:
from module.setting import instCpStockCode, instCpCodeMgr, instStockChart, instCpCybos, instCpTdUtil, instCPTd0311
from module import connection, get, save, search, update
from datetime import datetime
from datetime import timedelta
from sqlalchemy import create_engine
import pandas as pd
import pymysql
import pandas as pd
import os
import time

In [2]:
stock_list = ['A000020', 'A000040', 'A000050', 'A000060', 'A000070', 'A000100', 'A000120', 'A000140', 'A000150', 'A000180']

# 날짜 지정 필수
today = str(datetime.today().year) + str(datetime.today().month) + str((datetime.today() - timedelta(days=1)).day)
yesterday = str(datetime.today().year) + str(datetime.today().month) + str((datetime.today() - timedelta(days=2)).day)

# mysql connect하기 위한 아이디 비밀번호 포트 데이터베이스 등록 및 conn 리턴
def sqlalchemy_connect_ip(ip_address, db_name):
    engine = create_engine("mysql+pymysql://admin:"
                +"big15" # user password
                +"@{0}:3306/{1}?charset=utf8".format(ip_address, db_name)
                , encoding='utf8')
    
    return engine.connect()

def get_pymysql_db_list(conn, db_name):

    # 현재 DB 내 존재하는 테이블(종목) 추출
    sql = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = '{0}'".format(db_name)

    result = conn.execute(sql)
    db_list = [item[0] for item in result.fetchall()]

    return db_list

def get_investing_data(conn, yesterday, investing_data_list):

    investing_df = pd.DataFrame()

    # 현재 DB 내 존재하는 테이블(종목) 추출
    for table in investing_data_list:
        conn = sqlalchemy_connect_ip('192.168.50.123', 'investing_data')
        sql = "SELECT * FROM investing_data.`{0}` WHERE 날짜={1} ".format(table, yesterday)

        result = conn.execute(sql)

        temp_df = pd.DataFrame(result.fetchall())
        investing_df = pd.concat([investing_df, temp_df], axis=1)

    investing_df.drop(columns='날짜', axis=1, inplace=True) # 어제 날짜
    investing_df.rename(index={0:today}, inplace=True) # 오늘 날짜로 reindex
    
    return investing_df

def get_pymysql_day_stock(conn, code, yesterday, investing_df):

    code = code[-6:]
    sql = "SELECT 날짜, 전일대비, 상장주식수, 시가총액, 외국인현보유수량, 외국인현보유비율, 기관순매수량, 기관누적순매수량, 년, 월, 일 FROM stock_info.`{0}` WHERE 날짜={1} ORDER BY 시간 DESC LIMIT 1".format(code, yesterday)

    result = conn.execute(sql)

    target_df = pd.DataFrame(result.fetchall())
    target_df.set_index('날짜', inplace=True)
    target_df.rename(index={int(yesterday):today}, inplace=True)
    target_df = pd.concat([target_df, investing_df], axis=1)

    return target_df

def get_realtime_stock_info(code, today):
    
    instStockChart.SetInputValue(0, code) # 종목명
    instStockChart.SetInputValue(1, ord('2')) # 1 : 기간으로 요청, 2: 개수로 요청
    instStockChart.SetInputValue(3, today) # 요청 시작일
    instStockChart.SetInputValue(2, today) # 요청 종료일
    instStockChart.SetInputValue(4, 1) # 요청 개수
    '''
    # 요청할 데이터 종류(리스트 형태로 요청 가능)
    0 : 날짜, 1 : 시간 - hhmm, 2 : 시가, 3 : 고가, 4 : 저가, 5 : 종가
    6 : 전일대비, 7 : 없음, 8 : 거래량, 9 : 거래대금, 10 : 누적체결매도수량
    11 : 누적체결매수수량, 12 : 상장주식수, 13 : 시가총액, 14 : 외국인주문한도수량, 15 : 외국인주문가능수량
    16 : 외국인현보유수량, 17 : 외국인현보유비율, 18 : 수정주가일자, 19 : 수정주가비율, 20 : 기관순매수
    21 : 기관누적순매수, 22 : 등락주선, 23 : 등락비율, 24 : 예탁금, 25 : 주식회전율
    26 : 거래성립률
    '''
    instStockChart.SetInputValue(5, [0, 1, 2, 3, 4, 5, 8, 9, 10, 11])
    instStockChart.SetInputValue(6, ord('m')) # 'D' : 일봉, 'm' : 분봉
    instStockChart.SetInputValue(9, ord('1'))

    instStockChart.BlockRequest() # 위 정보로 요청

    numrow, numcolumn = instStockChart.GetHeaderValue(3), instStockChart.GetHeaderValue(2)

    index = []
    for i in range(numrow):
        index_ = str(instStockChart.GetDataValue(0,i))
        index.append(index_)

    stock_info = pd.DataFrame(columns=numcolumn[1:], index=index)

    for num in range(numrow):
        for col in range(len(numcolumn)):
            # 1,2,3,4,5,6,7,8,9, 10
            stock_info.iloc[num, col-1] = str(instStockChart.GetDataValue(col,num))

    if instCpCybos.GetLimitRemainCount(1) < 3:
        while True:
            if instCpCybos.GetLimitRemainCount(1) > 3:
                break
            time.sleep(0.1)

    return stock_info.iloc[0:1]


In [3]:

#################################################################################################
# Investing Data 생성, 1회 실행 후 추가 실행 필요 없음
#################################################################################################

sqlalchemy_conn = sqlalchemy_connect_ip('192.168.50.123', 'investing_data')
investing_data_list = get_pymysql_db_list(sqlalchemy_conn, 'investing_data')
investing_df = get_investing_data(sqlalchemy_conn, yesterday, investing_data_list)

#################################################################################################

In [5]:
# 실시간 감시 종목
while False:

    target_df = pd.DataFrame()
    for code in stock_list:
        print(code)

        day_stock_investing_df = get_pymysql_day_stock(sqlalchemy_conn, code, yesterday, investing_df)
        each_target_df = get_realtime_stock_info(code, today)
        each_target_df = pd.concat([each_target_df, day_stock_investing_df], axis=1)
        target_df = pd.concat([target_df, each_target_df], axis=0)

    # 모델 load 영역 #

    # 모델 predict 영역 #

    # predict 확률로 매수 진행

    

    




# 실행 종료 시 DB connection 종료 필요
# sqlalchemy_conn.close()

In [3]:
# 주문 초기화
# initCheck = instCpTdUtil.TradeInit(0)

# if (initCheck != 0):
#     print("주문 초기화 실패")
#     exit()


# 주식 매수 주문
acc = instCpTdUtil.AccountNumber[0] #계좌번호
accFlag = instCpTdUtil.GoodsList(acc, 1)  # 주식상품 구분
print(acc, accFlag[0])

instCPTd0311.SetInputValue(0, "2")   # 2: 매수
instCPTd0311.SetInputValue(1, acc )   #  계좌번호
instCPTd0311.SetInputValue(2, accFlag[0])   # 상품구분 - 주식 상품 중 첫번째
instCPTd0311.SetInputValue(3, "A005930")   # 종목코드 - 필요한 종목으로 변경 필요 
instCPTd0311.SetInputValue(4, 10)   # 매수수량 - 요청 수량으로 변경 필요 
instCPTd0311.SetInputValue(5, 10000)   # 주문단가 - 필요한 가격으로 변경 필요 
instCPTd0311.SetInputValue(7, "0")   # 주문 조건 구분 코드, 0: 기본 1: IOC 2:FOK
instCPTd0311.SetInputValue(8, "01")   # 주문호가 구분코드 - 01: 보통

# 매수 주문 요청
nRet = instCPTd0311.BlockRequest()
if (nRet != 0) :
    print("주문요청 오류", nRet)    
    # 0: 정상,  그 외 오류, 4: 주문요청제한 개수 초과 
    exit()

rqStatus = instCPTd0311.GetDibStatus()
errMsg = instCPTd0311.GetDibMsg1()
if rqStatus != 0:
    print("주문 실패: ", rqStatus, errMsg)
    exit()

com_error: (-2147352567, '예외가 발생했습니다.', (0, None, 'Module:CpDib\nObj:CpTdUtil\nFun:AccountNumber\nType:0\nDescription:TradeInit을 하지 않았습니다.', None, 0, -2147418113), None)